## Getting a bit more deeper into Agents

### Conversational ReAct Agents
This agent is designed for use in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [ ]:
%pip install langchain langchain_openai python-dotenv duckduckgo_search wikipedia

In [ ]:
from langchain.agents import create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')

In [ ]:
tools = load_tools(["llm-math"],  llm=llm)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,)

In [ ]:
output_1=conversational_agent.invoke("Add 7 to 9 and tell me the result")
output_2=conversational_agent.invoke("Add 5 to the previous result")

In [ ]:
print(output_1)
print(output_2)

## ReAct Docstore
This agent utilizes the React framework to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A `docstore` in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [1]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain_openai import OpenAI
from langchain.agents.react.base import DocstoreExplorer

In [3]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")]

#initializing the agent
llm = OpenAI(temperature=0)
docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=8
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.react.base.DocstoreExplorer` was deprecated in langchain 0.1.0 and will be removed in 0.2.0
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [4]:
docstore_agent.invoke({ "input" : "Tell me a few key things about Mahatma Gandhi." })



> Entering new AgentExecutor chain...
Thought: I need to search Mahatma Gandhi and find key information about him.
Action: Search[Mahatma Gandhi]
Observation: Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī; 2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London, and was called to the bar in June 1891, at the age of 22. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He we

{'input': 'Tell me a few key things about Mahatma Gandhi.',
 'output': 'Agent stopped due to iteration limit or time limit.'}

## Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions.

In [9]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="useful for when you need to search the internet for information"
    )

]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [10]:
print(agent.invoke("Question: Who was the president of USA when the first Moon landing took place?"))



> Entering new AgentExecutor chain...
 Yes.
Follow up: When did the first Moon landing take place?
Intermediate answer: Apollo 11, U.S. spaceflight in which astronauts Neil Armstrong and Buzz Aldrin became the first people to walk on the Moon. Apollo 11 was the culmination of the Apollo program and a massive national commitment by the United States to beat the Soviet Union in putting people on the Moon. Neil Armstrong (1930-2012) was a U.S. astronaut who became the first human to walk on the moon on July 20, 1969, as part of the Apollo 11 mission. Neil Armstrong was the first person to walk on the moon. He was an astronaut. He flew on two space missions. One was Apollo 11. That mission landed on the moon. ... What do you do for an encore when you've just completed the first human Moon landing mission? You go on a 38-day around the world goodwill tour, visiting 29 cities in 24 countries ... On 20 July 1969, the world watched in anticipation as Apollo 11's 'Eagle' lunar module touched 

The above code imports necessary modules from the LangChain library, including agents and language models. It sets up a conversational agent with a specific agent configuration called “self-ask-with-search”. The agent utilizes the “Intermediate Answer” tool, which performs Wikipedia searches.